In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Testing

Run the code below to compile the tests before running the tests in the above block! Comment out the tests you don't want to run.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma
import time
import math
import matplotlib as mp
import scipy as sp
import pylab as py
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation
import pandas as pd
import os
plt.rcParams['animation.html'] = 'html5'

# from ipynb.fs.full.NS_dynamics import *;
from ipynb.fs.full.NS_init import *;
from ipynb.fs.full.kernels import *;

Gauss:  0.32314708027098704
Starcrash:  -0.005152245421052439
Cubic Spline:  -0.011023163023913216
Wend 2:  0.32314708027098704
Wend 4:  0.32314708027098704


In [ ]:
def test_plot_kernels(init = True, keys = ["acc"],ylimits=False, hs=[0.01],ks = ["faber","cubic_spline","wendland_4","wendland_2","gaussian"],times = [(0.001,.02)],num_points=200,plot=False): #,(0.01,.2),(0.0001,.002),(.1,2),(1,20)] # (dt, tEnd)
    STYLE =  'dark_background'#'_classic_test_patch'#'seaborn-deep'
    plt.style.use(STYLE)
    KERNELS="-".join(np.copy(ks))
    VARS =""
    ylims = {"acc": (0,10), "dAdt": (0,10),"visc": (0,0.3),"pos":(-12,12),"rho":(0,4000),"grav":(0,0.01),"Pressure":(0,0.01),"(P - grav force)":(0,0.03)}
    ## Test initialization!
    kern_plot = {}
    for h in hs:
        print("\n ******* SUMMARY ******** \n")
        for time in times:
            for k in ks:
                filename="test_get_acc_h=" + str(h) + "dt="+ str(time[0])+ "End=" +str(time[1])
                #eos = EOS("polytropic")
                star1 = NS_("mystar", mass = 1, radius = 12, num_points = num_points,plot_initialized_star=False,kernel_type=k,prt=True)
                if init:
                    pass
                else:
                    star1 = NS_("mystar", mass = 1, radius = 13, num_points = 400,plot_initialized_star=False,kernel_type=kt)
                    star2 = NS_("mysecondstar", mass = 1, radius = 13, num_points = 400,plot_initialized_star=False,kernel_type=kt)
                    star2.com = [8,0,0]
                    star1.com = [-8,0,0]
                    combined_star = star1.combine(star2)
                    print(combined_star.num_points)
                    tEnd = 800
                    print(kt)
                    combined_star.kernel_type = kt
                    com_list = get_com_lists(R = 10)
                    combined_star.plot_init=True
                    combined_star.update_star_new(tag=kt,plotRealTime=True,dt = 0.5,COM_POSITIONS=com_list,params=params)
                kern_plot[k]= star1.test_acc_data
                if plot:
                    plot_single_timestep(star1.pos,title=k+" "+str(h))
        d = star1.test_acc_data
        for key in d.keys():
            if key in keys:
                data = []
                for k in ks:
                    data.append(kern_plot[k][key])   # append data for each kernel
                data = np.asarray(data)
                if ylimits:
                    plt.ylim(ylims[key])
                if key not in ['grav']:
                    csfont = {'fontname':'Times New Roman'}
                    hfont = {'fontname':'Helvetica'}
                    titles= {"acc":"Acceleration","rho":"Density","P":"Pressure","pos":"Position"}
                    k_titles= {"gaussian":"Gaussian","wendland_4":"Wendland 4","wendland_2":"Wendland 2","cubic_spline":"Cubic Spline","faber":"Faber"}
                    print("\nplotting ", titles[key])
                    for i in range(len(data)):
                        plt.plot(data[i])
                        TITLE=titles[key]#
                    plt.title(TITLE,loc='right',**csfont)
                    plt.legend([k_titles[i] for i in ks])
                    plt.ylabel(titles[key])
                    plt.xlabel("Timestep (Code Time)")
                    plt.savefig("plots/kernel/"+TITLE+", h="+str(h)+"-"+KERNELS+".jpeg", dpi=150)
                    plt.show()
                else:
                    pass
                if key in []:#["pos"]:
                    print("making movie")
                    plot(data, "pos_NS_anim_"+k)
                
#                     for i in range(len(data)):
#                         plt.plot(data[i])
#                     plt.title(key+",h="+str(h)+", tFin="+str(time[1]),loc='right')
#                     plt.legend(ks)
#                     plt.show()
                print("\n")
    return star1.test_acc_data

In [ ]:
def test_get_acc(keys = ["acc"],ylimits=False, hs=[0.01],ks = ["faber","cubic_spline","wendland_4","wendland_2","gaussian"],times = [(0.001,.02)],num_points=200,plot=False): #,(0.01,.2),(0.0001,.002),(.1,2),(1,20)] # (dt, tEnd)
    ylims = {"acc": (0,10), "dAdt": (0,10),"visc": (0,0.3),"pos":(-12,12),"rho":(0,4000),"grav":(0,0.01),"Pressure":(0,0.01),"(P - grav force)":(0,0.03)}
    ## Test initialization!
    kern_plot = {}
    for h in hs:
        print("\n ******* SUMMARY ******** \n")
        for time in times:
            for k in ks:
                filename="test_get_acc_h=" + str(h) + "dt="+ str(time[0])+ "End=" +str(time[1])
                eos = EOS("polytropic")
                star1 = NS("star-test-acc", eos, None, num_points=num_points,h_init=h,dt=time[0],tEnd=time[1],test = True,kernel_type=k)
                
                result = star1.initialization_notes
                print("Star with initial smoothing length " + str(h)+ " and kernel '" + str(k) + "' \n - was initialized and " + result + "\n")
                kern_plot[k]= star1.TEST_ACC_DATA
                if plot:
                    plot_single_timestep(star1.pos,title=k+" "+str(h))
        d = star1.TEST_ACC_DATA
        for key in d.keys():
            if key in keys:
                data = []
                for k in ks:
                    data.append(kern_plot[k][key])   # append data for each kernel
                data = np.asarray(data)
                if ylimits:
                    plt.ylim(ylims[key])
                if key not in ['visc','grav']:
                    print("plotting ", key)
                    for i in range(len(data)):
                        plt.plot(data[i])
                    plt.title(key+",    h="+str(h)+", tFin="+str(time[1]),loc='right')
                    plt.legend(ks)
                    plt.show()
                else:
                    pass
                if key in []:#["pos"]:
                    print("making movie")
                    plot(data, "pos_NS_anim_"+k)
                
#                     for i in range(len(data)):
#                         plt.plot(data[i])
#                     plt.title(key+",h="+str(h)+", tFin="+str(time[1]),loc='right')
#                     plt.legend(ks)
#                     plt.show()
                print("\n")
    return star1.TEST_ACC_DATA
 

In [3]:
def test_kernels():
    eos = EOS("polytropic")
    kernels = ["wend2","wend4", "gaussian", "cubic_spline"]
    passed = 0
    x = np.asarray([[0,1,1.1,30,2,1.2,1.9,1.999]])
    for grad in [False,True]:
        for tag in kernels:
            try:
                star = NS("star1", eos, kernel_type=tag)
            except:
                print("You are having a problem initializing a star.")
                return passed,len(kernels)*10
            print("testing kernel implementation...")
            for h in [0.1,0.01,0.001,1,10]:
                solution = TESTW(x,x,x,h,tag=tag,grad=grad)
                if not grad:
                    g ="evaluation"
                else:
                    g = "gradient"
                if solution:
                    passed +=1
                else:
                    print(tag + " kernel failed testing: "+g+" for h = "+str(h)+".")

    return passed,len(kernels)*10

def test_density():
    return 0,0

def test_get_acc(): 
    return 0,0


def single_star_test0():
    date = "Nov17_2023"
    trial = "4"
    notes = "NS_TEST_eta800_polytrop"
    spec = notes
    folder = ""
    eos_tag = "polytropic"
    dt = 0.001 # seconds
    tEnd = 1 # seconds
    m = 3 # SMs
    r = 10 # km
    points_ = 300 # sph points
    nu_ = 1 # artificial viscosity
    lt = "vanilla" # Lagrangian formulation tag
    kt = "wendland_2" # Kernel type
    eta = 0.08
    cl = -1 # scale factor for lambda, an artificial potential energy.
    simulate_NS(eos_tag,dt,tEnd,m,r,points_,nu_,lt,kt,date,trial,notes=notes,specs = spec,folder_name=folder,eta_=eta,cl=cl)
    
def combine_star_test():
    eos_tag = "polytropic"
    dt = 0.001 # seconds
    tEnd = 1 # seconds
    m = 3 # SMs
    r = 10 # km
    points_ = 300 # sph points
    nu_ = 1 # artificial viscosity
    lt = "vanilla" # Lagrangian formulation tag
    kt = "wendland_2" # Kernel type
    eta = 0.08
    cl = -1 # scale factor for lambda, an artificial potential energy.
    simulate_NS(eos_tag,dt,tEnd,m,r,points_,nu_,lt,kt,date,trial,notes=notes,specs = spec,folder_name=folder,eta_=eta,cl=cl)
    
def flyby_test():
    pass


def all_tests(): 
    p,t = test_kernel()
    print("kernel passed" + str(p)+"/"+str(t) + " tests")
    p,t = test_density()
    print("test_density passed" + str(p)+"/"+str(t) + " tests")
    p,t = test_get_acc()
    print("test_get_acc passed" + str(p)+"/"+str(t) + " tests")
    single_star_test()
    print("single star test: complete!")
    p,t = combine_star_test()
    print("combine star test: complete!")
    
def plot_init_test():
    print("testing plot init.")
    # test sphere plot above!!
    N = 1000
    r = 1
    m=mesh_pts(N,r)
    print("meshed points: ",len(m))
    #print(N/len(m))

    # Import libraries
    from mpl_toolkits import mplot3d
    import numpy as np
    import matplotlib.pyplot as plt

    p=m
    print("Creating figure...")
    fig = plt.figure(figsize = (10, 7))
    ax = plt.axes(projection ="3d")
    x = p[:,0]
    y = p[:,1]
    z = p[:,2]
    # Creating plot
    ax.scatter3D(x, y, z, color = "green")
    plt.title("simple 3D scatter plot")
    # show plot
    plt.show()
    
def single_star_test1():
    print("Testing simulate.")
    print("single_star_test1")
    eos = EOS('ross2018')
    my_ns1 = NS("star_1", eos, num_points = 700,eta = 2e-2);
    #my_ns2 = NS("star_2", eos, num_points = 200,eta = 1e-1);
    testing_params = {"dt": 10, "distance apart": 45}
    simulate_NS([my_ns1],test_values = testing_params, testing = True, folder_name = "test-simulate-single-star-output")



In [1]:
## test gradW implementation
def test_kernels2():
    print("test gradW")
    pos = np.asarray([[0.01,2,3],[10,1.2,4],[1,100,1],[0,0,0]])
    dx,dy,dz = getPairwiseSeparations(pos,pos)
    h = 5
    k = "faber"
    print(gradW( dx, dy, dz, h , k))
    
    print("test getDensity")
    r =  np.asarray([[0.8607446,0.8607446,0.8607446]])
    pos =  np.asarray([[0.860744,0,1]])
    m =  0.01
    h = np.asarray([[11]])
    print(getDensity(r, pos, m, h ))
    print("test complete")
      

def testNS_init():
    ## Test initialization!
    hs = [0.001,0.01]#,0.1,1,2,10,20]
    times = [(0.001,.02)]#,(0.01,.2),(0.0001,.002),(.1,2),(1,20)] # (dt, tEnd)
    ks = ["faber","cubic_spline","wendland_4"]
    for h in hs:
        for time in times:
            for k in ks:
                print("_______\n\n\n\ndt/ending time: ", time,"Kernel: ",k,"h: ",h,"\n\n")
                filename="wendland4_test_init_h=" + str(h) + "dt="+ str(time[0])+ "End=" +str(time[1])
                star1 = NS("star1", "polytropic",None, num_points=200,h_init=h,dt=time[0],tEnd=time[1],test = False,kernel_type=k)
                result = star1.initialization_notes
                print("Star with smoothing length " + str(h)+ " and kernel " + str(k) + " was initialized and " + result)
                print("\n\n\n\n***********************************************************************************")
                
def testNS():
    ## Test initialization!
    hs = [0.001,0.01]#,0.1,1,2,10,20]
    times = [(0.01,.2)]#,(0.01,.2),(0.0001,.002),(.1,2),(1,20)] # (dt, tEnd)
    ks = ["faber","cubic_spline","wendland_4"]
    passed = 0
    for h in hs:
        for time in times:
            for k in ks:
                try:
                    filename="wendland4_test_init_h=" + str(h) + "dt="+ str(time[0])+ "End=" +str(time[1])
                    star1 = NS("star1", "polytropic",None, num_points=150,h_init=h,dt=time[0],tEnd=time[1],test = False,kernel_type=k)
                    print("\nPASS: Succesfully initialized a star with kernel '" + str(k) + "' and smoothing length " + str(h))
                    passed += 1
                except:
                    print("\n Failed to initialize a star with kernel '" + str(k) + "' and smoothing length " + str(h))
                    pass
    ntests = len(hs)*len(ks)*len(times)
    print(" TESTS PASSED: " + str(passed) + "/" + str(ntests))